In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-06-18 02:00:32--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.48MB/s    in 0.2s    

2022-06-18 02:00:32 (5.48 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|                Y|
|R12VVR0WH5Q24V|          5|            0|          0|   N|                Y|
| RNCCKB6TV5EEF|          4|            0|          0|   N|                Y|
|R2NZXYIVCGB13W|          5|            0|          6|   N|                Y|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|                Y|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|                Y|
|R12ENYLFGGNWRV|          5|            1|          1|   N|                Y|
|R2R07E5PNXEUO5|          4|            0|          0|   N|                Y|
|R27BA52AKWMWN3|          5|            0|          0|   N|                Y|
| RLF8DOID2KD5O|          3|            0|          0|   N|     

In [6]:
# Create a new DF to retrieve all the rows where the 'total_votes' count is equal
# to or greater than 20 
vine_greater_df = vine_df.filter(vine_df["total_votes"] >= 20)
vine_greater_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [7]:
# Create a new DF that retrievs all rows where the number of 'helpful_votes' divided by
# 'total_votes' is equal to or greater than 50%
divided_vine_df = vine_greater_df.filter(vine_greater_df["helpful_votes"] / vine_greater_df["total_votes"] >= 0.5 )
divided_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [8]:
# Create a DF that retrieves all the rows where a review was written as part of the Vine program (paid)
paid_vine_df = divided_vine_df.filter(divided_vine_df["vine"] == "Y" )
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|
|R2XRYNV2SY3ZKL|          5|           53|         56|   Y|                N|
|R1Y93KWKAX1P5N|          2|           26|         31|   Y|                N|
|R2QVTDYYLTP8SL|          5|           21|         24|   Y|     

In [9]:
# Create DF that retrieves all the rows where the review was not part of the Vine program (unpaid)
unpaid_vine_df = divided_vine_df.filter(divided_vine_df["vine"] == "N" )
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [11]:
# determine the total number of reviews, number of 5-star reviews, and the % of 5-star
# reviews for paid vs unpaid.
total_num_review_paid = paid_vine_df.count()
five_star_review_paid = paid_vine_df.filter(paid_vine_df["star_rating"] == 5).count()
percentage_five_star_review_paid = round((five_star_review_paid / total_num_review_paid) * 100, 2)
print("The total number of reviews for vine reviewer is",total_num_review_paid )
print("The total 5-star reviews for vine reviewer is",five_star_review_paid )
print("The percentage for 5-star reviews for vine reviewer is",percentage_five_star_review_paid, "%")

The total number of reviews for vine reviewer is 22
The total 5-star reviews for vine reviewer is 13
The percentage for 5-star reviews for vine reviewer is 59.09 %


In [12]:
total_num_review_unpaid = unpaid_vine_df.count()
five_star_review_unpaid = unpaid_vine_df.filter(unpaid_vine_df["star_rating"] == 5).count()
percentage_five_star_review_unpaid = round((five_star_review_unpaid / total_num_review_unpaid) * 100, 2)
print("The total number of reviews for non-vine reviewer is",total_num_review_unpaid )
print("The total 5-star reviews for non-vine reviewer is",five_star_review_unpaid )
print("The percentage for 5-star reviews for non-vine reviewer is",percentage_five_star_review_unpaid, "%")

The total number of reviews for non-vine reviewer is 26987
The total 5-star reviews for non-vine reviewer is 14475
The percentage for 5-star reviews for non-vine reviewer is 53.64 %


In [13]:
#total number of review
print("Total number of reviews for both vine and non-vine reviewer is ",total_num_review_paid + total_num_review_unpaid )

Total number of reviews for both vine and non-vine reviewer is  27009
